In [1]:
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
import re
import nltk
import requests
import pyLDAvis
import pyLDAvis.gensim_models
from nltk.stem import WordNetLemmatizer
import spacy
import numpy as np
import os

# Make sure to download nltk stopwords if not already
nltk.download("stopwords")
nltk.download("punkt_tab")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dnaso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dnaso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dnaso\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# FOR T3 Dataset

In [2]:
# RUN FOR 1800 DATASET
dataset_1800 = "../T3_Transcripts"
tags_1800_dir = "../video_tags"
title_dir = "../video_titles"

filename_pattern = re.compile(r"^(.*?)\.txt$")

data_records = []
tag = []
video_title = ""

for file in sorted(os.listdir(dataset_1800)): 
        match = filename_pattern.match(file)
        if match:
            video_id = match.group(1)  # Extract Video Id and Title
            file_path = os.path.join(dataset_1800, file)

             # Read transcript content
            with open(file_path, "r", encoding="utf-8") as f:
                transcript = f.read().strip()

            tag_file = os.path.join(tags_1800_dir, f"{video_id}.txt")

            tag = []

            try:
                with open(tag_file, "r", encoding="utf-8") as f:
                    tag.append(f.read().strip())
            except FileNotFoundError:
                print(f"Files for video ID {video_id} not found, skipping.")

            title_file = os.path.join(title_dir, f"{video_id}.txt")

            try:
                with open(title_file, "r", encoding="utf-8") as f:
                    video_title = f.read().strip()
            except FileNotFoundError:
                print(f"Files for video ID {video_id} not found, skipping.")

            data_records.append({
                "Video Id": video_id,
                "Video Title": video_title,
                "Tags": tag,  # Ensures it's always a list
                "Transcript": transcript
            })


# Convert to DataFrame
data = pd.DataFrame(data_records)

data["text"] = data["Tags"].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x)) + " " + data["Transcript"]


# Display summary
print(f"Loaded {len(data)} transcripts into data.")
print(data.head())

Loaded 2704 transcripts into data.
      Video Id                                        Video Title  \
0  --8n6A8Q6M0      $200 Luxury Beach Hotel in The Philippines 🇵🇭   
1  -1B7cVoZr1c  Marine reacts to the Philippine Light Reaction...   
2  -7vF5F-1btE  Ultimate Filipino Food Festival In The Netherl...   
3  -9bfDHHneyU         SHOWING MY SISTER SB19 'GENTO' Music Video   
4  -C5iB25BRsA  10 Reasons/Do not Retire TO the Philippines/Mo...   

                                                Tags  \
0  [luke, luke damant, damant, travel, traveler, ...   
1  [philippines, philippine, filipino, lrr, light...   
2  [ultimate filipino food festival, filipino foo...   
3  [sb19 gento reaction, sb19 gento reaction vide...   
4  [se asia, cost of living in the philippines yo...   

                                          Transcript  \
0  Oh, my God.\nIt is just the perfect recipe for...   
1  in November of 2000. The Philippine authorized...   
2  welcome back to pinoy planet we're going t

In [3]:
# def is_not_filipino_word(word):
#     # Process the word using SpaCy
#     doc = nlp(word)
#     # Use the word vector from SpaCy to determine if it's a Filipino word
#     embedding = doc.vector
#     return np.linalg.norm(embedding) < 0.25  # This threshold might need adjustment

In [4]:
# Step 2: Text Preprocessing
stop_words = set(stopwords.words("english"))

# nlp = spacy.load("tl_fasttext_transition")

# Function to fetch stopwords from GitHub URL
def fetch_stopwords_from_github(url):
    response = requests.get(url)
    github_stopwords = response.text.splitlines()  # Split by new lines
    return set(github_stopwords)

# GitHub URL for stopwords
github_stopwords_url = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-en/master/stopwords-en.txt'
github_stopwords = fetch_stopwords_from_github(github_stopwords_url)

with open("../stop_words.txt", "r", encoding="utf-8") as file:
    custom_stop_words = set(file.read().splitlines())

stop_words.update(custom_stop_words, github_stopwords)

# custom_stop_words = ['like', 'yeah', 'know', 'um', 'uh', 'really', 'one', 'go', 'right', 'okay', 'well', 'said', 
#                      'going', 'got', 'na', 'always', 'every', 'each', 'say', 'el', 'little', 'still', 
#                      'best', 'dutch', 'nice', 'great', 'awesome', 'good', 'cool', 'love', 'amazing', 'wow' ]
# broad_terms = ['philippines', 'philippine', 'british', 'filipino', 'video', 'http', 'korea', 'korean', 
#                'youtube', 'google', 'united', 'america', 'american']
# kpop_keywords = ['kpop', '필리핀', 'bts', 'blackpink', 'twice', 'exo', 'k-pop', 'seventeen', 
#                  'stray kids', 'nct', 'kdrama', 'aespa', 'taehyung', 'jimin', 'jungkook']
# more_keywords = [
#     'breaking news', 'report', 'coverage', 'investigation', 'interview', 'documentary', 
#     'journalist', 'headline', 'reporter', 'current events', 'special report', 
#     'analysis', 'documented', 'broadcast', 'reporting', 'v', 'food', 'travel', 'react', 
#     'reacts', 'reaction', 'foreigner', 'thing', 'visit', 'dc', 'japan', 'first', 'fast', 
#     'asia', 'ang', 'indian', 'thai', 'vietnamese', 'russia', 'gon', 'canada', 'canadian', 'russian', 
#     'russia', 'guy', 'lot', 'bit', 'diba', 'ola', 'cuz', 'thai', 'thailand', 'person', 'citizen', 'foreigner', 'foreign', 'foreigners',
#     'facebook', 'filipinos', 'filipinas', 'vlog', 'vlogs', 'vlogging', 'hashtag', 'india', 'bro', 'dito', 'people', 'time', 'music', 'gonna', 'life', 
#     'lol', 'guys', 'tho', 'cute', 'hmm', 'huh', 'channel', 'subscribe', 'day6', 'mandarin', 'chinese', 'beautiful',
#     'chuckles', 'fbe', 'hit', 'laughs', 'yo', 'ka', 'word', 'living', 'boi', 'minimum', 'ya', 'successful', 'perfectly', 'yeap', 
#     'wondering', 'fantastic', 'hurry', 'german', 'age', 'country', 'subscribing', 'bluesy', 'jump', 'pretty', 'understanding', 'personalized',
#     'and', 'the', 'to', 'of', 'in', 'for', 'on', 'with', 'at', 'by', 'from', 'up', 'about', 'over', 'into', 'through', 'between', 'under', 'against', 'all',
#     'you', 'haha', 'hahaha', 'ha', 'hey', 'bye', 'hello', 'hi', 'oh', 'blah', 'easy', 'alright', 'ta', 'day', 'ooh', 'en', 'do', 'lot', 'comment', 'notification', 
#     'korean', 'jjajangmyeon', 'jajangmyeon', 'damn', 'yall', 'month', 'week', 'year', 'ohhh', 'pvf', 'dude', 'mmm', 'kagilagilalas', 'ofcourse', 'australia', 'uxo', 
#     'atleast', 'yusuf', 'bangkok', 'ot', 'anytime', 'allover', 'kala', 'nope', 'wan', 'brazil', 'smooth', 'ot', 'timeshere', 'batchof', 'yep', 'opo', 'del',
#     'gosh', 'po', 'ourself', 'wo', 'wait', 'ugh', 'nyc', 'whoa', 'nicaragua', 'yup', 'em', 'bout', 'le', 'omg', 'overwhelm', 'maam', 'nicer', 'haha', 'hahaha', 'ha', 
#     'nbcs', 'lana', 'rc', 'whatsoever', 'oxy', 'decade', 'whyd', 'unknown', 'ahhhhh', 'ohoh', 'ohto', 'ohhhh', 'bruh', 'ooe', 'ahmedabad', 'mexico', 
#     'understand', 'excuse', 'kinda', 'applause', 'oooh', 'thiswhat', 'nevermind', 'ahh', 'againthank', 'toto', 'aww', 'nah', 'bbmas', 'ay', 'op', 'huh', 'huhu',
#     'tada', 'beacuse', 'voila', 'upstairs', 'thatswhy', 'yea', 'that', 'armenia', 'or', 'not', 'funwhat', 'aka', 'armeniathat', 'woosexy', 'worth', 'laugh', 'box', 
#     'xd', 'vb', 'eff', 'ananya', 'welsh', 'latron', 'shout', 'whatwhat', 'what', 'pause', 'why', 'thats', 'byebye', 'iv', 'bye', 'chance', 'aware', 'edge', 'type_', 'true',
#     'yay', 'woo', 'shit', 'fucking', 'fuck' 
# ]
# stop_words.update(custom_stop_words, kpop_keywords, broad_terms, more_keywords, github_stopwords)

lemmatizer = WordNetLemmatizer()


def preprocess_text(text):
    # Lowercase and remove non-alphabet characters
    text = re.sub(r'[^a-z0-9\s]', '', text.lower())
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords and filter out short words
    words = [word for word in words if word not in stop_words and len(word) > 1 and word.isalpha()]

    # for word in words:
    #     if is_not_filipino_word(word):
    #         words.remove(word)

    final_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(final_words)


data["cleaned_text"] = data["text"].apply(preprocess_text)

# remove pos tags
# the only problem with this approach is that using the standard english model can sometimes remove words like ube, bgc, lechon believing it to be a part of speech
nlp = spacy.load("en_core_web_sm")

# ADV, VERB seems decent
pos_tags = {'VERB', 'ADV', 'PRON',  'PART'}

def remove_pos_tags(text):
    doc = nlp(text)
    filtered_words = []
    for token in doc:
        if token.pos_ not in pos_tags:
            filtered_words.append(token.text)
        else:
            print(token.text + " " + token.pos_)
    return " ".join(filtered_words)


data["cleaned_text"] = data["cleaned_text"].apply(remove_pos_tags)


# tfidf = TfidfVectorizer(max_df=0.8, min_df=2, stop_words=custom_stop_words)
# temp = data["cleaned_text"]
# X = tfidf.fit_transform(temp)

# vocabulary = tfidf.get_feature_names_out()
# vocabulary

# def filter_words(text):
#     words = word_tokenize(text) # Split text into words
#     print(words)
#     filtered_words = [word for word in words if word in vocabulary] 
#     print(filtered_words) # Keep only words in vocab
#     return " ".join(filtered_words)  # Join back into a string

# data["tfidf"] = data["cleaned_text"].apply(filter_words)


C:\Users\dnaso\AppData\Roaming\Python\Python311\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


recipe VERB
honestly ADV
arrived VERB
raining VERB
absolutely ADV
overlooking VERB
basically ADV
beached VERB
expanding VERB
enjoying VERB
explore VERB
setting VERB
whatnot ADV
heading VERB
honestly ADV
swimming VERB
arrived VERB
walking VERB
walking VERB
noticed VERB
require VERB
whatnot ADV
honestly ADV
stayed VERB
overlooking VERB
honestly ADV
beat VERB
stoked VERB
honestly ADV
view VERB
serving VERB
guessing VERB
filming VERB
bridesmaid VERB
bridesmaid VERB
understands VERB
breaking VERB
add VERB
frozen VERB
walking VERB
literally ADV
literally ADV
climate VERB
whatnot ADV
stay VERB
absolutely ADV
expect VERB
paying VERB
absolutely ADV
normalized VERB
fortunately ADV
shower VERB
shower VERB
pushed VERB
pushed VERB
functioning VERB
enables VERB
simulate VERB
importantly ADV
stayed VERB
raving VERB
approve VERB
coming VERB
sit VERB
checked VERB
left VERB
comfy VERB
cushioning VERB
sitting VERB
sitting VERB
watch VERB
sponsored VERB
paying VERB
heading VERB
drying VERB
surrounded VERB

In [5]:
# import calamancy
# for model in calamancy.models():
#     print(model)

# nlp = calamancy.load("tl_calamancy_lg-0.2.0")

# def extract_entities(text):
#     doc = nlp(text)
#     entities = [ent.text for ent in doc.ents]
#     scrapped_words = [token.text for token in doc if token.ent_type_ == ""]
#     return entities, scrapped_words

# # Apply the function to each item in data["tfidf"]
# data["entities"], data["scrapped"] = zip(*data["tfidf"].apply(extract_entities))

# # Display the first few rows to verify
# print(data.head())

# # Write scrapped words to a text file
# with open('scrapped_words_fil_2.txt', 'w', encoding='utf-8') as file:
#     for scrapped in data['scrapped']:
#         file.write(" ".join(scrapped) + '\n')

# for item in data["entities"]:
#     print(item

# REMOVE TOP FREQ WORDS

In [6]:
from collections import Counter

# Step 1: Build word frequency dictionary
all_words = []
for row in data['cleaned_text']:
    words = row.split()  # Assuming words are space-separated
    all_words.extend(words)

word_freq = Counter(all_words)
top_100_words = set([word for word, _ in word_freq.most_common(100)])

# Step 2: Remove top 100 words from each row
def remove_top_words(text, stopwords):
    return ' '.join([word for word in text.split() if word not in stopwords])

data['cleaned_text'] = data['cleaned_text'].apply(lambda x: remove_top_words(x, top_100_words))

In [7]:

# Created File Using only Cleaned Text
with open('T3-minus-freq.txt', 'w', encoding='utf-8') as file:
    for row in data['cleaned_text']:
        file.write(row + '\n')


# to use HLTM:
# java -cp HLTA.jar;HLTA-deps.jar tm.hlta.HTD "file name of text" "output name"

In [8]:


print(data.loc[data["Video Id"] == "3Wn0qDg4uDQ", "cleaned_text"].values[0])

mukbang item menu menu staff friendly jolly jollybees mix pineapple burger brownish milky color salam ala poe mendele chain jollybees ton eating menu dining pie pie pineapple pineapple pineapple pineapple fine hamburger burger burger burger burger spaghetti float recommend sandwich sandwich meal sandwich fry everybodys staff friendly jolly jolly fee pick staff boomin everybodys fine meet pick spray paint deeper safe meal salamat poh spaghetti jali gravy pineapple juice float coke float burger mystery jali fry burger dig spaghetti spaghetti chain serf spaghetti clean fork spoon dig mix spaghetti sausage mistaken spaghetti sausage spaghetti spaghetti sausage cook fan spaghetti mix spaghetti balance saucy dry pineapple juice juice pineapple juice organic somethings curious fat fat burger southwest massive massive mullet vegetable onion lettuce bun bite hotter hotter hotter tongue flavor lie flavor popeyes kfc kfc sandwich popeye sandwich jolly sandwich spaghetti pineapple juice pineapple 